## From Local Mode to Cluster Mode
Spark provides 3 options working on a cluster.
<img src="images/spark_modes.png">

MESOS and YARN are for sharing the spark cluster between teams. So we will stick with Standalone mode. 

With big data the data is too big to fit on the single computer so it is kept on clusters. As a data scientist you'll run the spark jobs on the data stored in external database or a third-party storage rented from a Cloud computing provider like Amazon.

<img src="images/spark_bigdata.png">

To build a spark cluster you have to options:
1. Buy computers and build cluster 
2. Use cloud platforms like amazon web services and rent a cluster of machines and expand or shrink the cluster size as you need. Just login from anywhere to use the clusters.

Our setup will look like this:

<img src="images/rented_spark_cluster.png">

The Data will be stored on S3 storage and then machines for spark will be rented using EC2 service of AWS services. And then we'll login to the spark cluster remotely and submit the job to the cluster. 

## Setup Instructions AWS
If you want to create a spark cluster manually you can follow this [guide]( https://blog.insightdatascience.com/spinning-up-a-spark-cluster-on-spot-instances-step-by-step-e8ed14ebb3b). However its quite tedious and you have to perform same steps for multiple machines and if you have to update something you have to do it several times. Fortunately AWS offers an easier option called Elastic Map Reduce or EMR for short. EMR provides you EC2 instances with big data technologies installed.
Now following are the instructions to setup EMR cluster:
1. Create ssh key pair to securely connect to the cluster. To do this go to the EC2 service. Select **`Key pairs`** in **`NETWORK & SECURITY`** and create a key pair. Named it as `spark-cluster` and a `pem` file will be downloaded for you. 
2. Go to EMR service and create a cluster by naming it and according to the requirements. Make sure to select `Spark` in **`Software configuration`** section. Select the instance type according to your requirements. 
To compare different EC2 instance type either go [here](https://aws.amazon.com/ec2/instance-types/) or [here](https://ec2instances.info/). Select EC2 key pair and create cluster.
For more follow this [tutorial](https://www.youtube.com/watch?v=ZVdAEMGDFdo) on youtube.

## Using Notebooks on your Cluster
After the cluster is in `Waiting` state connect to the cluster. Amazon has multiple ways to connect to the cluster. We'll use `Notebook` by clicking on the left side in the menu. Click on `create notebook`. Then name your notebook and attach a cluster to it and leave the rest to default and create the cluster. For more elaboration following this [tutorial](https://www.youtube.com/watch?v=EcIYPkCkehY) from **Udacity** on youtube.

## Spark Scipts
Up until now jupyter notebooks were used. They have the following advantages:
1. Good for prototyping
2. Exploring and visualizing the data
3. Easily share the results with others

But Jupyter notebooks are not good for automating the workflows. That's where scipts come in to play. For more elaboration following this [tutorial](https://www.youtube.com/watch?v=bfOocPv54EI) from **Udacity** on Youtube.

## Submitting Spark Scripts

In [2]:
%%writefile lower_scripts.py
from pyspark.sql import SparkSession

if __name__ == '__main__':
    """
        example program to show how to submit applications
    """
    spark = SparkSession\
            .builder\
            .appName('LowerSongTitles')\
            .getOrCreate()

    log_of_songs = [
        "Despacito",
        "Nice for what",
        "No tears left to cry",
        "Despacito",
        "Havana",
        "In my feelings",
        "Nice for what",
        "Despacito",
        "All the stars"
    ]
    
    distributed_song_log = spark.sparkContext.parallelize(log_of_songs)
    
    print(distributed_song_log.map(lambda x:x.lower()).collect())
    
    spark.stop()

Overwriting lower_scripts.py


change the pem file permissions using the follwing linux command:

    chmod 600 spark-cluster.pem
and connect to the master node. Use the following command to run the spark job:

    spark-submit --master yarn ./lower_script.py

## Creating EMR Cluster

In [2]:
import boto3
import pandas as pd

In [7]:
credentials = pd.read_csv('credentials/credentials.csv')
ACCESS_KEY = credentials['Access key ID'][0]
SECRET_ACCESS_KEY = credentials['Secret access key'][0]

In [13]:
emr = boto3.client('emr',
                   region_name = 'us-west-2',
                   aws_access_key_id = ACCESS_KEY,
                   aws_secret_access_key = SECRET_ACCESS_KEY)

ec2 = boto3.resource('ec2', 
                     region_name = 'us-west-2',
                     aws_access_key_id = ACCESS_KEY,
                     aws_secret_access_key = SECRET_ACCESS_KEY)
client = boto3.client('ec2',
                      region_name = 'us-west-2',
                      aws_access_key_id = ACCESS_KEY,
                      aws_secret_access_key = SECRET_ACCESS_KEY)

In [61]:
VPC = client.describe_vpcs()['Vpcs'][0]['VpcId']
subnet = client.create_subnet(CidrBlock='172.31.0.0/16',VpcId=VPC)
SUBNET = subnet['Subnet']['SubnetId']

In [62]:
cluster_id = emr.run_job_flow(
    Name='test_emr_job_boto3',
    LogUri='s3://naqeeb-emr-test/logs',
    ReleaseLabel='emr-5.28.0',
    Applications=[
        {
            'Name': 'Spark'
        },
    ],
    Instances={
        'InstanceGroups': [
            {
                'Name': "Master nodes",
                'Market': 'ON_DEMAND',
                'InstanceRole': 'MASTER',
                'InstanceType': 'm5.xlarge',
                'InstanceCount': 1,
            },
            {
                'Name': "Slave nodes",
                'Market': 'ON_DEMAND',
                'InstanceRole': 'CORE',
                'InstanceType': 'm5.xlarge',
                'InstanceCount': 3,
            }
        ],
        'Ec2KeyName': 'spark-cluster',
        'KeepJobFlowAliveWhenNoSteps': True,
        'TerminationProtected': False,
        'Ec2SubnetId': SUBNET,
    },
    VisibleToAllUsers=True,
    JobFlowRole='EMR_EC2_DefaultRole',
    ServiceRole='EMR_DefaultRole'
)

print ('cluster created with the step...', cluster_id['JobFlowId'])

cluster created with the step... j-H53TSJFINU0U


## Reading and Writing to Amazon S3

In [66]:
emr.terminate_job_flows(JobFlowIds=[
        cluster_id['JobFlowId'],
    ])

{'ResponseMetadata': {'RequestId': 'b01b8c69-3158-4d18-9162-8567cb3eae93',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b01b8c69-3158-4d18-9162-8567cb3eae93',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 08 Jan 2020 17:27:04 GMT'},
  'RetryAttempts': 0}}

In [68]:
client.delete_subnet(SubnetId=SUBNET)

{'ResponseMetadata': {'RequestId': 'b0487ccf-583f-4881-a6a2-8b6afaaf2fec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'content-length': '225',
   'date': 'Wed, 08 Jan 2020 17:29:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}